<a href="https://colab.research.google.com/github/inhyek/newsest_insta/blob/master/dessert_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth, drive
auth.authenticate_user()
drive.mount('/content/gdrive')

COLAB_ROOT='/content/gdrive/My Drive/Colab Notebooks/'
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/

import os
os.listdir(COLAB_ROOT)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 connect-gdrive.ipynb					        PyTorch
 Dessert						        RAdam
 Obama							        Severstal
'Out-of-distribution_detection_via_Bayesian_CNN.ipynb의 사본'


['Severstal',
 'connect-gdrive.ipynb',
 'RAdam',
 'Obama',
 'Out-of-distribution_detection_via_Bayesian_CNN.ipynb의 사본',
 'PyTorch',
 'Dessert']

In [0]:
DATA_ROOT = COLAB_ROOT + 'Dessert/data/'

In [0]:
!pip install --upgrade pip
!pip uninstall -y tensorflow
!pip install tensorflow-gpu
# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time
import json

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

print(tf.__version__)

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
The following GPU devices are available: /device:GPU:0
2.0.0


In [0]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

In [0]:
# module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" 
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3) 
    return img

In [0]:
YEARS = ['2016', '2019']
CLASSES = ['bread', 'cake', 'macaron', 'icecream']

ORG_PATH = DATA_ROOT + 'original/'
CROP_PATH = DATA_ROOT + 'crop/'
if not os.path.isdir(CROP_PATH):
    os.makedirs(CROP_PATH)

SUBCLASS_JSON = DATA_ROOT + 'subclass.json'
DONE = {}

def init_subclass():
    SUBCLASS = {
        'bread': set(),
        'cake': set(),
        'macaron': set(),
        'icecream': set()
    }
    if os.path.exists(SUBCLASS_JSON):
        with open(SUBCLASS_JSON, 'r') as f:
            SUBCLASS = json.load(f)
        for key in SUBCLASS:
            SUBCLASS[key] = set(SUBCLASS[key])

    for cls in CLASSES:
        if len(SUBCLASS[cls]) != 0:
            continue
        for year in YEARS:
            print(year, cls)
            cls_path = '{}{}/{}'.format(ORG_PATH, year, cls)
            cls_zip = '{}{}/{}.zip'.format(ORG_PATH, year, cls)
            !rm -rf tmp target.zip; mkdir tmp
            !cp '$cls_zip' target.zip
            !unzip -q target.zip -d tmp/
            files = os.listdir('tmp/')
            total_len = len(files)
            start_time = time.time()
            for cnt, f in enumerate(files):
                try:
                    img = load_img('tmp/{}'.format(f))
                    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
                    result = detector(converted_img)
                    result = {key:value.numpy() for key,value in result.items()}
                    for i in range(len(result['detection_class_entities'])):
                        if result["detection_scores"][i] < 0.2:
                            continue
                        SUBCLASS[cls].add(result['detection_class_entities'][i].decode('ascii'))
                except Exception as e:
                    print(f)
                    print(e)
                if cnt%30 == 0:
                    elapsed_time = time.time() - start_time
                    expected_time = elapsed_time / (cnt+1) * total_len
                    print('{}/{}, ({}sec/{}sec)'.format(cnt, total_len, int(elapsed_time), int(expected_time)))
        
        dump = {}
        for key in SUBCLASS:
            dump[key] = list(SUBCLASS[key])
        with open(SUBCLASS_JSON, 'w') as f:
            json.dump(dump, f, indent=4)

    return SUBCLASS

def load_subclass():
    SUBCLASS = {}
    with open(SUBCLASS_JSON, 'r') as f:
        loaded = json.load(f)
    for key in loaded:
        SUBCLASS[key] = set(loaded[key])
    return SUBCLASS

SUBCLASS = init_subclass()
# SUBCLASS = load_subclass()



2016 cake
0/7438, (0sec/2646sec)
30/7438, (10sec/2629sec)
60/7438, (21sec/2608sec)
90/7438, (32sec/2623sec)
120/7438, (42sec/2627sec)
150/7438, (53sec/2627sec)
180/7438, (64sec/2634sec)
210/7438, (74sec/2635sec)
240/7438, (85sec/2631sec)
270/7438, (96sec/2635sec)
300/7438, (106sec/2636sec)
330/7438, (117sec/2635sec)
360/7438, (127sec/2635sec)
390/7438, (138sec/2632sec)
420/7438, (148sec/2631sec)
2016cake3010.jpg
Expected image (JPEG, PNG, or GIF), got unknown format starting with '\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000' [Op:DecodeJpeg]
450/7438, (159sec/2625sec)
480/7438, (169sec/2625sec)
510/7438, (180sec/2626sec)
540/7438, (191sec/2626sec)
570/7438, (201sec/2625sec)
600/7438, (212sec/2626sec)
630/7438, (222sec/2626sec)
660/7438, (233sec/2627sec)
690/7438, (244sec/2627sec)
720/7438, (254sec/2628sec)
750/7438, (265sec/2629sec)
780/7438, (276sec/2628sec)
810/7438, (286sec/2628sec)
840/7438, (297sec/2628sec)
870/7438, (307sec/2628sec)
900/7438, (318sec/2628sec)


In [0]:
for year in YEARS:
    for cls in CLASSES:
        print(year, cls)
        start_time = time.time()
        org_zip = '{}/{}/{}.zip'.format(ORG_PATH, year, cls)
        crop_zip = '{}_{}.zip'.format(year, cls)
        if os.path.exists('{}{}'.format(CROP_PATH, crop_zip)):
            continue
        !rm -rf tmp result target.zip; mkdir tmp; mkdir result
        !cp '$org_zip' target.zip
        !unzip -q target.zip -d tmp/
        files = os.listdir('tmp/')
        total_len = len(files)
        for cnt, f in enumerate(files):
            try:
                img = load_img('tmp/{}'.format(f))
                converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
                result = detector(converted_img)
                
                idxs = tf.image.non_max_suppression(result["detection_boxes"], 
                                                    result["detection_scores"], max_output_size=100)
        
                boxes = tf.gather(result["detection_boxes"], idxs).numpy()
                entities = tf.gather(result["detection_class_entities"], idxs).numpy()
                scores = tf.gather(result["detection_scores"], idxs).numpy()
                target_idxs = []
                img_numpy = img.numpy()
                H, W, C = img_numpy.shape
                num = 0
                for i in range(idxs.shape[0]):
                    cand = entities[i].decode('ascii')
                    if cand not in SUBCLASS[cls]:
                        continue
                    if scores[i] < 0.2:
                        continue
                    cropped_img = img_numpy[int(boxes[i][0]*H):int(boxes[i][2]*H), 
                                            int(boxes[i][1]*W):int(boxes[i][3]*W)]
                    Image.fromarray(cropped_img).save('result/{}-{}-{}.jpg'.format(os.path.splitext(f)[0], num, cand))
                    num+=1
            except Exception as e:
                print(f)
                print(e)
            if cnt%30 == 0:
                    elapsed_time = time.time() - start_time
                    expected_time = elapsed_time / (cnt+1) * total_len
                    print('{}/{}, ({}sec/{}sec)'.format(cnt, total_len, int(elapsed_time), int(expected_time)))
        
        !cd result; zip -q '$crop_zip' ./*; mv '$crop_zip' '$CROP_PATH'
            # image_with_boxes = draw_boxes(img.numpy(),
            #                               boxes[target_idxs],
            #                               entities[target_idxs],
            #                               scores[target_idxs])
            # display_image(image_with_boxes)
            

2016 bread
2016 cake
2016 macaron
2016 icecream
2019 bread
2019 cake
2019 macaron
2019 icecream
0/15916, (44sec/710858sec)
30/15916, (57sec/29376sec)
